In [6]:
import requests
import pandas as pd
import matplotlib.pyplot as plt

BASE_URL = "http://127.0.0.1:8000"


In [7]:
ticker = "AAPL"

earnings_dates = [
    "2024-01-31",
    "2024-04-30",
    "2024-07-30",
    "2024-10-30",
    "2025-01-28",
]


In [8]:
rows = []

for call_date in earnings_dates:
    try:
        resp = requests.get(
            f"{BASE_URL}/features/earnings",
            params={"ticker": ticker, "call_date": call_date},
            timeout=10,
        )
        resp.raise_for_status()
        data = resp.json()

        # API returns: {"features": [ { ... } ]}
        features_list = data.get("features", [])
        if not features_list:
            print(f"No features returned for {ticker} {call_date}")
            continue

        row = features_list[0]
        rows.append(row)
        print(f"OK: {ticker} {call_date}")

    except Exception as e:
        print(f"Error for {ticker} {call_date}: {e}")

len(rows)


Error for AAPL 2024-01-31: 404 Client Error: Not Found for url: http://127.0.0.1:8000/features/earnings?ticker=AAPL&call_date=2024-01-31
Error for AAPL 2024-04-30: 404 Client Error: Not Found for url: http://127.0.0.1:8000/features/earnings?ticker=AAPL&call_date=2024-04-30
Error for AAPL 2024-07-30: 404 Client Error: Not Found for url: http://127.0.0.1:8000/features/earnings?ticker=AAPL&call_date=2024-07-30
Error for AAPL 2024-10-30: 404 Client Error: Not Found for url: http://127.0.0.1:8000/features/earnings?ticker=AAPL&call_date=2024-10-30
OK: AAPL 2025-01-28


1

In [5]:
df = pd.DataFrame(rows)
df


,ticker,call_date,token_count,sentiment_mean,sentiment_std,uncertainty_score,return_1d,return_3d,return_5d
0,AAPL,2025-01-28,55,0.049234,0.12221,0.072727,0.004617,-0.009485,-0.04302
